1. 이미지 텐서 변환

In [33]:
import os
import imageio.v2 as imageio
import torch

# imageio 라이브러리로 이미지를 불러오면 NumPy 배열 형태로 반환된다.
img_arr = imageio.imread("../_00_data/a_image-dog/bobby.jpg")
print(img_arr.shape) # (Height, Width, Channel) 순서

# NumPy 배열을 파이토치 텐서로 변환한다.
img = torch.from_numpy(img_arr)

# 텐서의 차원 순서를 (H, W, C)에서 (C, H, W)로 변경한다.
out = img.permute(2, 0, 1)
print(out.shape)

# 여러 이미지 파일을 리스트로 관리한다.
data_dir = "../_00_data/b_image-cats"
filenames = [name for name in os.listdir(data_dir) if name.endswith('.png')]

# batch_size 만큼의 이미지를 담을 빈 텐서를 생성한다.
# shape: (Batch, Channel, Height, Width)
batch_size = 3
batch = torch.zeros(batch_size, 3, 256, 256, dtype=torch.uint8)

# 각 이미지를 불러와 (C, H, W) 형태로 변환한 뒤, 배치 텐서에 순서대로 담는다.
for i, filename in enumerate(filenames):
  img_arr = imageio.imread(os.path.join(data_dir, filename))
  img_t = torch.from_numpy(img_arr)
  img_t = img_t.permute(2, 0, 1)
  batch[i] = img_t

print(batch.shape)

# 정규화를 위해 텐서의 자료형을 float으로 변경한다.
batch = batch.float()
# 모든 픽셀 값을 255.0으로 나누어 0.0 ~ 1.0 사이의 값으로 만든다.
batch /= 255.0

# 각 채널별로 평균과 표준편차를 계산한다.
n_channels = batch.shape[1]
for c in range(n_channels):
  mean = torch.mean(batch[:, c])
  std = torch.std(batch[:, c])
  # 각 채널의 데이터를 (데이터 - 평균) / 표준편차 공식으로 표준화한다.
  batch[:, c] = (batch[:, c] - mean) / std

(720, 1280, 3)
torch.Size([3, 720, 1280])
torch.Size([3, 3, 256, 256])


이미지 라이브러리는 주로 데이터를 HWC (Height, Width, Channel) 순서의 배열로 불러온다.

파이토치는 일반적으로 CHW (Channel, Height, Width) 순서를 사용하므로 permute 함수로 차원 순서를 변경해야 한다.

여러 이미지를 한 번에 처리하기 위해 BCHW (Batch, Channel, Height, Width) 형태의 배치 텐서를 구성한다.

정규화는 텐서의 값 범위를 일정한 수준으로 맞추는 과정이다. 코드에서는 픽셀 값을 0과 1 사이로 조정하고, 이어서 각 채널이 평균 0, 표준편차 1을 갖도록 표준화했다.

2. tabular 와인 데이터

In [34]:
import csv
import os
import numpy as np

# 경로를 조합하여 CSV 파일 위치를 지정한다.
wine_path = "../_00_data/d_tabular-wine/winequality-white.csv"

# numpy.loadtxt: 파일을 읽어 NumPy 배열로 만든다.
# delimiter=";": 세미콜론을 기준으로 데이터를 나눈다.
# skiprows=1: 첫 번째 행(헤더)을 건너뛴다.
wineq_numpy = np.loadtxt(wine_path, dtype=np.float32, delimiter=";", skiprows=1)

# csv 모듈을 사용해 헤더 행만 따로 읽어온다.
col_list = next(csv.reader(open(wine_path), delimiter=';'))
print(col_list)

# torch.from_numpy: NumPy 배열을 파이토치 텐서로 변환한다.
# 이 과정은 메모리를 공유하므로 변환 속도가 빠르다.
wineq = torch.from_numpy(wineq_numpy)
print(wineq.shape)

['fixed acidity', 'volatile acidity', 'citric acid', 'residual sugar', 'chlorides', 'free sulfur dioxide', 'total sulfur dioxide', 'density', 'pH', 'sulphates', 'alcohol', 'quality']
torch.Size([4898, 12])


numpy.loadtxt는 숫자 데이터로 구성된 텍스트 파일을 불러오는 데 사용된다. 헤더가 있는 경우 skiprows=1로 제외할 수 있다.

torch.from_numpy는 NumPy 배열을 메모리 복사 없이 텐서로 변환한다.

In [35]:
# 데이터 분리: 마지막 열을 제외한 모든 데이터를 입력(data)으로 선택
data = wineq[:, :-1]
# 타겟 분리: 마지막 열만 정답(target)으로 선택
target = wineq[:, -1]

# 타겟의 자료형을 정수형으로 변환
target = target.to(torch.int64)
print("Data Shape:", data.shape)
print("Target Shape:", target.shape)

# 원-핫 인코딩
# 10x10 크기의 단위 행렬을 생성한다.
eye_matrix = torch.eye(10)
# target 텐서의 값을 인덱스로 사용하여 단위 행렬의 행을 선택한다.
# 이를 통해 각 정수 레이블이 원-핫 벡터로 변환된다.
onehot_target = eye_matrix[target]
print("One-hot Target Shape:", onehot_target.shape)

# 데이터 표준화
# dim=0: 각 열(특성)별로 평균과 분산을 계산한다.
data_mean = torch.mean(data, dim=0)
data_var = torch.var(data, dim=0)
# (데이터 - 평균) / 표준편차 공식으로 표준화를 진행한다.
data = (data - data_mean) / torch.sqrt(data_var)

Data Shape: torch.Size([4898, 11])
Target Shape: torch.Size([4898])
One-hot Target Shape: torch.Size([4898, 10])


원-핫 인코딩은 범주형 데이터를 0과 1로 구성된 벡터로 변환하는 기법이다. torch.eye와 정수 인덱싱을 이용해 구현할 수 있다.

표준화는 각 특성의 평균을 0, 분산을 1로 맞추어 데이터의 스케일을 통일하는 과정이다.

In [36]:
from sklearn.model_selection import train_test_split

# 데이터를 학습셋(80%)과 테스트셋(20%)으로 분할한다.
X_train, X_test, y_train, y_test = train_test_split(data, onehot_target, test_size=0.2)
print("Train data shape:", X_train.shape)
print("Test data shape:", X_test.shape)

# 위에서 실행한 모든 데이터 처리 과정을 하나의 함수로 정의한다.
def get_wine_data():
  wine_path = "../_00_data/d_tabular-wine/winequality-white.csv"
  wineq_numpy = np.loadtxt(wine_path, dtype=np.float32, delimiter=";", skiprows=1)
  wineq = torch.from_numpy(wineq_numpy)

  data = wineq[:, :-1]
  target = wineq[:, -1].to(torch.int64)

  eye_matrix = torch.eye(10)
  onehot_target = eye_matrix[target]

  data_mean = torch.mean(data, dim=0)
  data_var = torch.var(data, dim=0)
  data = (data - data_mean) / torch.sqrt(data_var)

  X_train, X_valid, y_train, y_valid = train_test_split(data, onehot_target, test_size=0.2)

  return X_train, X_valid, y_train, y_valid

Train data shape: torch.Size([3918, 11])
Test data shape: torch.Size([980, 11])


sklearn.model_selection.train_test_split은 데이터를 무작위로 섞어 지정된 비율로 분할하는 함수이다.

데이터 로딩부터 전처리, 분할까지의 전체 파이프라인을 하나의 함수로 만들어두면 코드의 구조가 명확해지고 재사용이 쉬워진다.

3. tabular 캘리포니아 집 값 

In [37]:
import torch
from sklearn.datasets import fetch_california_housing

# 데이터셋을 불러온다.
housing = fetch_california_housing()
print(housing.keys())

# 데이터는 NumPy 배열 형태이며, shape과 특성 이름을 확인한다.
print(housing.data.shape)
print(housing.feature_names)

# 타겟(주택 가격)의 shape과 이름을 확인한다.
print(housing.target.shape)
print(housing.target_names)

dict_keys(['data', 'target', 'frame', 'target_names', 'feature_names', 'DESCR'])
(20640, 8)
['MedInc', 'HouseAge', 'AveRooms', 'AveBedrms', 'Population', 'AveOccup', 'Latitude', 'Longitude']
(20640,)
['MedHouseVal']


sklearn.datasets는 여러 표준 데이터셋을 불러오는 함수를 제공한다.

fetch_california_housing이 반환하는 객체의 .data 속성은 입력 특성을, .target 속성은 정답 값을 담고 있는 NumPy 배열이다.

In [38]:
import numpy as np

# 각 특성(열)별로 평균과 분산을 계산한다. axis=0은 열 기준 연산을 의미한다.
data_mean = np.mean(housing.data, axis=0)
data_var = np.var(housing.data, axis=0)

# (데이터 - 평균) / 표준편차(분산의 제곱근) 공식으로 표준화를 수행한다.
data = (housing.data - data_mean) / np.sqrt(data_var)
target = housing.target

표준화는 데이터의 각 특성이 평균 0, 분산 1을 갖도록 변환하는 과정이다.

이 단계까지의 모든 연산은 넘파이를 사용하여 수행되었다.

In [39]:
from sklearn.model_selection import train_test_split

# 데이터를 학습셋(80%)과 테스트셋(20%)으로 분할한다.
X_train, X_test, y_train, y_test = train_test_split(data, target, test_size=0.2)

# 분할된 넘파이 배열들을 파이토치 텐서로 변환한다.
X_train = torch.from_numpy(X_train)
X_test = torch.from_numpy(X_test)
y_train = torch.from_numpy(y_train)
y_test = torch.from_numpy(y_test)

print("Train data shape:", X_train.shape)
print("Test data shape:", X_test.shape)

Train data shape: torch.Size([16512, 8])
Test data shape: torch.Size([4128, 8])


sklearn.model_selection.train_test_split 함수는 데이터셋을 지정된 비율로 무작위 분할한다.

데이터 준비의 마지막 단계에서 torch.from_numpy를 통해 모든 데이터를 파이토치 텐서로 변환하여 이후 과정을 준비한다.

4. 공용 자전거

In [40]:
import os
import numpy as np
import torch

torch.set_printoptions(edgeitems=2, threshold=50, linewidth=75)

bikes_path = "../_00_data/e_time-series-bike-sharing-dataset/hour-fixed.csv"
# converters: 1번 열(날짜) 데이터에서 '일'만 추출하여 float으로 변환한다.
bikes_numpy = np.loadtxt(
  fname=bikes_path, dtype=np.float32, delimiter=",", skiprows=1,
  converters={ 1: lambda x: float(x[8:10]) }
)
bikes = torch.from_numpy(bikes_numpy).to(torch.float)

# view(-1, 24, ...): 전체 시간 데이터를 24시간 단위(하루)로 묶는다.
# (-1은 자동으로 계산되는 일 수를 의미)
daily_bikes = bikes.view(-1, 24, bikes.shape[1])
print("Reshaped data shape:", daily_bikes.shape)

# 마지막 열을 타겟으로, 나머지를 데이터로 분리한다.
daily_bikes_data = daily_bikes[:, :, :-1]
daily_bikes_target = daily_bikes[:, :, -1].unsqueeze(dim=-1)

print("Data shape:", daily_bikes_data.shape)
print("Target shape:", daily_bikes_target.shape)

Reshaped data shape: torch.Size([730, 24, 17])
Data shape: torch.Size([730, 24, 16])
Target shape: torch.Size([730, 24, 1])


numpy.loadtxt의 converters는 파일 로딩 중 특정 열에 함수를 적용하여 데이터를 맞춤형으로 처리하는 기능이다.

view(-1, 24, ...)는 연속적인 시계열 데이터를 일정한 시간 단위로 묶어 배치 차원을 생성하는 표준적인 기법이다.

In [41]:
# 첫째 날의 날씨 데이터(9번 열)를 선택
first_day_data = daily_bikes_data[0]
weather_column = first_day_data[:, 9]

# 날씨 정보는 1, 2, 3, 4로 기록되어 있으나 인덱스는 0부터 시작하므로 1을 빼준다.
weather_int = weather_column.to(torch.int64) - 1

# 4x4 단위 행렬을 생성
eye_matrix = torch.eye(4)
# 날씨 값을 인덱스로 사용하여 단위 행렬에서 해당하는 행(원-핫 벡터)을 추출한다.
weather_onehot = eye_matrix[weather_int]

# 기존 데이터에 원-핫 인코딩된 날씨 정보를 추가한다.
first_day_data_torch = torch.cat(tensors=(first_day_data, weather_onehot), dim=1)
print("Shape after one-hot encoding:", first_day_data_torch.shape)

# 전체 데이터셋에 대해 위의 과정을 반복한다.
day_data_torch_list = []
for daily_idx in range(daily_bikes_data.shape[0]):
  day = daily_bikes_data[daily_idx]
  weather_onehot = eye_matrix[day[:, 9].to(torch.int64) - 1]
  day_data_torch = torch.cat(tensors=(day, weather_onehot), dim=1)
  day_data_torch_list.append(day_data_torch)

# 리스트에 담긴 각 날의 텐서들을 stack 함수로 합쳐 다시 하나의 텐서로 만든다.
daily_bikes_data = torch.stack(day_data_torch_list, dim=0)
print("Final data shape:", daily_bikes_data.shape)

Shape after one-hot encoding: torch.Size([24, 20])
Final data shape: torch.Size([730, 24, 20])


1부터 시작하는 범주형 데이터를 인덱스로 사용하려면 1을 빼서 0부터 시작하도록 조정해야 한다.

torch.cat을 사용하여 기존 특성 행렬에 새로 생성된 원-핫 인코딩 특성 행렬을 옆으로 이어 붙일 수 있다.

In [42]:
# 기존의 범주형 날씨 열(9번)과 불필요한 instant 열(0번)을 제거한다.
# 1~8번 열과 10번 이후 열을 붙여 두 열을 제외시킨다.
daily_bikes_data = torch.cat(
  [daily_bikes_data[:, :, 1:9], daily_bikes_data[:, :, 10:]], dim=2
)
print("Shape after dropping columns:", daily_bikes_data.shape)

# 온도 특성(현재 8번 열)을 표준화한다.
temperatures = daily_bikes_data[:, :, 8]
# 전체 온도 데이터의 평균과 표준편차를 구해 표준화를 수행한다.
daily_bikes_data[:, :, 8] = (temperatures - torch.mean(temperatures)) / torch.std(temperatures)

Shape after dropping columns: torch.Size([730, 24, 18])


원-핫 인코딩을 적용한 후에는 원래의 범주형 열을 제거하여 중복 정보를 없애는 것이 일반적이다.

특정 특성을 정규화할 때는 해당 특성의 전체 분포를 고려하여 평균과 표준편차를 계산해야 한다.

5. 선형 회귀 데이터셋 로더 

In [43]:
import torch
from torch.utils.data import Dataset, DataLoader, random_split


class LinearRegressionDataset(Dataset):
  # 1. __init__: 데이터셋을 초기화하고 데이터를 생성하거나 불러온다.
  def __init__(self, N=50, m=-3, b=2, *args, **kwargs):
    super().__init__(*args, **kwargs)
    self.x = torch.rand(N, 2)
    self.noise = torch.rand(N) * 0.2
    self.m = m
    self.b = b
    self.y = (torch.sum(self.x * self.m) + self.b + self.noise).unsqueeze(-1)

  # 2. __len__: 데이터셋의 전체 샘플 개수를 반환한다.
  def __len__(self):
    return len(self.x)

  # 3. __getitem__: 주어진 인덱스(idx)에 해당하는 샘플 1개를 반환한다.
  def __getitem__(self, idx):
    return self.x[idx], self.y[idx]

  def __str__(self):
    str = "Data Size: {0}, Input Shape: {1}, Target Shape: {2}".format(
      len(self.x), self.x.shape, self.y.shape
    )
    return str

Dataset 클래스는 데이터가 저장된 위치와 개별 데이터를 가져오는 방법을 정의하는 일종의 명세서이다.

__init__에서 모든 데이터를 메모리에 올리거나, 데이터 파일의 경로 목록을 준비한다.

__len__은 데이터셋의 크기를 알려준다.

__getitem__은 인덱스를 받아 해당하는 데이터와 레이블 쌍을 반환한다. DataLoader가 이 메소드를 내부적으로 호출하여 데이터를 가져온다.

In [44]:
if __name__ == "__main__":
  # Dataset 객체를 생성한다.
  linear_regression_dataset = LinearRegressionDataset()
  print(linear_regression_dataset)

  # random_split: Dataset을 주어진 비율에 따라 무작위로 분할한다.
  # 여기서는 70% 학습용, 20% 검증용, 10% 테스트용으로 나눈다.
  train_dataset, validation_dataset, test_dataset = random_split(
      linear_regression_dataset, [0.7, 0.2, 0.1]
  )
  print("Split sizes:", len(train_dataset), len(validation_dataset), len(test_dataset))

  # DataLoader: Dataset으로부터 데이터를 배치 단위로 가져오는 반복자(iterator)를 생성한다.
  # batch_size: 한 번에 가져올 샘플의 개수
  # shuffle=True: 데이터를 가져오기 전에 매번 순서를 섞는다.
  train_data_loader = DataLoader(
    dataset=train_dataset,
    batch_size=4,
    shuffle=True
  )

  # DataLoader를 통해 배치 단위로 데이터를 순회한다.
  for idx, batch in enumerate(train_data_loader):
    input, target = batch
    print(f"Batch {idx} - Input Shape: {input.shape}, Target Shape: {target.shape}")

Data Size: 50, Input Shape: torch.Size([50, 2]), Target Shape: torch.Size([50, 1])
Split sizes: 35 10 5
Batch 0 - Input Shape: torch.Size([4, 2]), Target Shape: torch.Size([4, 1])
Batch 1 - Input Shape: torch.Size([4, 2]), Target Shape: torch.Size([4, 1])
Batch 2 - Input Shape: torch.Size([4, 2]), Target Shape: torch.Size([4, 1])
Batch 3 - Input Shape: torch.Size([4, 2]), Target Shape: torch.Size([4, 1])
Batch 4 - Input Shape: torch.Size([4, 2]), Target Shape: torch.Size([4, 1])
Batch 5 - Input Shape: torch.Size([4, 2]), Target Shape: torch.Size([4, 1])
Batch 6 - Input Shape: torch.Size([4, 2]), Target Shape: torch.Size([4, 1])
Batch 7 - Input Shape: torch.Size([4, 2]), Target Shape: torch.Size([4, 1])
Batch 8 - Input Shape: torch.Size([3, 2]), Target Shape: torch.Size([3, 1])


Dataset은 데이터셋 전체를, DataLoader는 그 데이터셋에서 데이터를 어떤 방식으로 가져올지를 정의한다.

random_split은 인덱스를 기준으로 데이터를 나누므로, 실제 데이터 복사 없이 효율적으로 데이터셋을 분할한다.

6. 2d 이미지 데이터 로더 

In [45]:
import os
import torch
from PIL import Image
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision import transforms

class DogCat2DImageDataset(Dataset):
  def __init__(self):
    # transforms.Compose: 여러 변환 단계를 순서대로 실행하는 파이프라인을 생성한다.
    self.image_transforms = transforms.Compose([
      # transforms.Resize: 모든 이미지의 크기를 (256, 256)으로 통일한다.
      transforms.Resize(size=(256, 256)),
      # transforms.ToTensor: PIL Image를 텐서로 변환하고, 픽셀 값을 [0, 1]로 정규화하며,
      # 차원 순서를 HWC에서 CHW로 변경한다.
      transforms.ToTensor()
    ])
    # 각 이미지 파일의 경로를 지정한다.
    dogs_dir = "../_00_data/a_image-dog"
    cats_dir = "../_00_data/b_image-cats"

    # PIL.Image.open으로 각 이미지를 불러와 리스트에 담는다.
    image_lst = [
      Image.open(os.path.join(dogs_dir, "bobby.jpg")),
      Image.open(os.path.join(cats_dir, "cat1.png")),
      Image.open(os.path.join(cats_dir, "cat2.png")),
      Image.open(os.path.join(cats_dir, "cat3.png"))
    ]

    # 리스트의 모든 이미지에 대해 정의된 변환을 적용한다.
    image_lst = [self.image_transforms(img) for img in image_lst]
    # 변환된 이미지 텐서 리스트를 stack 함수로 합쳐 하나의 배치 텐서로 만든다.
    self.images = torch.stack(image_lst, dim=0)

    # 각 이미지에 해당하는 레이블 텐서를 생성한다. (0: dog, 1: cat)
    self.image_labels = torch.tensor([[0], [1], [1], [1]])

  def __len__(self):
    return len(self.images)

  def __getitem__(self, idx):
    return self.images[idx], self.image_labels[idx]

  def __str__(self):
    return f"Data Size: {len(self.images)}, Input Shape: {self.images.shape}"

transforms.Resize는 배치로 묶을 모든 이미지의 크기를 동일하게 맞추는 필수 전처리 단계이다.

transforms.ToTensor는 이미지 데이터를 파이토치가 다룰 수 있는 형태인 CHW 순서의 텐서로 변환하고 값을 [0, 1] 범위로 조정한다.

Dataset의 __init__ 단계에서 모든 데이터를 불러오고 변환까지 마치는 방식을 사용할 수 있다. 이 경우, 데이터셋이 메모리에 모두 로드된다.

torch.stack은 동일한 크기를 가진 여러 텐서를 새로운 차원으로 결합하여 하나의 텐서로 만든다.

In [46]:
if __name__ == "__main__":
  dog_cat_2d_image_dataset = DogCat2DImageDataset()
  print(dog_cat_2d_image_dataset)

  # 데이터를 7:3 비율로 학습셋과 테스트셋으로 분할한다.
  train_dataset, test_dataset = random_split(dog_cat_2d_image_dataset, [0.7, 0.3])
  print("Split sizes:", len(train_dataset), len(test_dataset))

  # 학습셋에 대한 DataLoader를 생성한다.
  train_data_loader = DataLoader(
    dataset=train_dataset,
    batch_size=2,
    shuffle=True
  )

  # DataLoader를 통해 배치 단위로 데이터를 가져온다.
  for idx, batch in enumerate(train_data_loader):
    input, target = batch
    print(f"Batch {idx} - Input Shape: {input.shape}, Target Shape: {target.shape}")

Data Size: 4, Input Shape: torch.Size([4, 3, 256, 256])
Split sizes: 3 1
Batch 0 - Input Shape: torch.Size([2, 3, 256, 256]), Target Shape: torch.Size([2, 1])
Batch 1 - Input Shape: torch.Size([1, 3, 256, 256]), Target Shape: torch.Size([1, 1])


torchvision.transforms, Dataset, DataLoader는 이미지 데이터를 처리하는 파이토치의 표준적인 파이프라인 구성 요소이다.

transforms는 개별 데이터의 전처리를, Dataset은 데이터셋 전체의 구성을, DataLoader는 배치 단위 데이터 공급을 담당한다.

In [47]:
import os
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader, random_split


class WineDataset(Dataset):
  def __init__(self):
    # __init__ 메소드에서 데이터 로딩부터 변환, 전처리까지 모두 수행한다.
    wine_path = "../_00_data/d_tabular-wine/winequality-white.csv"
    wineq_numpy = np.loadtxt(wine_path, dtype=np.float32, delimiter=";", skiprows=1)
    wineq = torch.from_numpy(wineq_numpy)

    # 특성 데이터를 분리하고 표준화한다.
    data = wineq[:, :-1]
    data_mean = torch.mean(data, dim=0)
    data_var = torch.var(data, dim=0)
    self.data = (data - data_mean) / torch.sqrt(data_var)

    # 타겟 데이터를 분리하고 원-핫 인코딩한다.
    target = wineq[:, -1].to(torch.int64)
    eye_matrix = torch.eye(10)
    self.target = eye_matrix[target]

    # 데이터와 타겟의 샘플 수가 일치하는지 확인한다.
    assert len(self.data) == len(self.target)

  def __len__(self):
    return len(self.data)

  def __getitem__(self, idx):
    return self.data[idx], self.target[idx]

  def __str__(self):
    return f"Data Size: {len(self.data)}, Input Shape: {self.data.shape}"

데이터셋 생성에 필요한 모든 로직을 Dataset 클래스 내부에 캡슐화하면 코드의 재사용성과 가독성이 향상된다.

__init__에서 모든 데이터를 처리하여 self.data와 self.target에 저장하고, __getitem__에서는 단순히 해당 인덱스의 데이터를 반환하도록 구조를 단순화할 수 있다.

In [48]:
if __name__ == "__main__":
  wine_dataset = WineDataset()
  print(wine_dataset)

  # 데이터셋을 7:2:1 비율로 분할한다.
  train_dataset, validation_dataset, test_dataset = random_split(wine_dataset, [0.7, 0.2, 0.1])
  print("Split sizes:", len(train_dataset), len(validation_dataset), len(test_dataset))

  # DataLoader를 생성한다.
  train_data_loader = DataLoader(
    dataset=train_dataset,
    batch_size=32,
    shuffle=True,
    # drop_last=True: 마지막 배치의 크기가 batch_size보다 작으면 해당 배치를 버린다.
    drop_last=True
  )

  # DataLoader를 통해 배치 단위로 데이터를 순회한다.
  for idx, batch in enumerate(train_data_loader):
    input, target = batch
    print(f"Batch {idx} - Input Shape: {input.shape}, Target Shape: {target.shape}")

Data Size: 4898, Input Shape: torch.Size([4898, 11])
Split sizes: 3429 980 489
Batch 0 - Input Shape: torch.Size([32, 11]), Target Shape: torch.Size([32, 10])
Batch 1 - Input Shape: torch.Size([32, 11]), Target Shape: torch.Size([32, 10])
Batch 2 - Input Shape: torch.Size([32, 11]), Target Shape: torch.Size([32, 10])
Batch 3 - Input Shape: torch.Size([32, 11]), Target Shape: torch.Size([32, 10])
Batch 4 - Input Shape: torch.Size([32, 11]), Target Shape: torch.Size([32, 10])
Batch 5 - Input Shape: torch.Size([32, 11]), Target Shape: torch.Size([32, 10])
Batch 6 - Input Shape: torch.Size([32, 11]), Target Shape: torch.Size([32, 10])
Batch 7 - Input Shape: torch.Size([32, 11]), Target Shape: torch.Size([32, 10])
Batch 8 - Input Shape: torch.Size([32, 11]), Target Shape: torch.Size([32, 10])
Batch 9 - Input Shape: torch.Size([32, 11]), Target Shape: torch.Size([32, 10])
Batch 10 - Input Shape: torch.Size([32, 11]), Target Shape: torch.Size([32, 10])
Batch 11 - Input Shape: torch.Size([32, 

drop_last=True 옵션은 전체 데이터셋 크기가 배치 크기로 나누어 떨어지지 않을 때 사용된다.

이 옵션을 활성화하면 모든 배치의 크기가 batch_size로 동일하게 유지된다. len(train_dataset)인 3429를 32로 나누면 몫은 107이고 나머지는 5이다. drop_last=True이므로 나머지 5개 데이터로 구성된 마지막 배치는 버려지고, 총 107개의 배치만 생성된다.

7. 캘리포니아 집 값

In [49]:
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader, random_split


class CaliforniaHousingDataset(Dataset):
  def __init__(self):
    # scikit-learn에서 데이터셋을 불러온다.
    from sklearn.datasets import fetch_california_housing
    housing = fetch_california_housing()

    # NumPy를 사용하여 데이터 특성을 표준화한다.
    data_mean = np.mean(housing.data, axis=0)
    data_var = np.var(housing.data, axis=0)
    data_numpy = (housing.data - data_mean) / np.sqrt(data_var)

    # 전처리된 NumPy 배열을 PyTorch 텐서로 변환한다.
    self.data = torch.tensor(data_numpy, dtype=torch.float32)
    # 타겟 데이터도 텐서로 변환하고, unsqueeze로 차원을 추가한다.
    self.target = torch.tensor(housing.target, dtype=torch.float32).unsqueeze(dim=-1)

  def __len__(self):
    return len(self.data)

  def __getitem__(self, idx):
    return self.data[idx], self.target[idx]

  def __str__(self):
    return f"Data Size: {len(self.data)}, Input Shape: {self.data.shape}"

데이터 준비에 필요한 모든 코드를 Dataset 클래스의 __init__ 메소드에 포함시켜 로직을 구조화했다.

이 데이터셋의 타겟은 연속적인 숫자 값이므로 원-핫 인코딩 대신 float32 타입으로 변환하고 unsqueeze를 사용하여 (샘플 수, 1) 형태의 shape으로 만들었다.

In [50]:
if __name__ == "__main__":
  california_housing_dataset = CaliforniaHousingDataset()
  print(california_housing_dataset)

  # 데이터셋을 7:2:1 비율로 학습, 검증, 테스트용으로 분할한다.
  train_dataset, validation_dataset, test_dataset = random_split(
      california_housing_dataset, [0.7, 0.2, 0.1]
  )
  print("Split sizes:", len(train_dataset), len(validation_dataset), len(test_dataset))

  # DataLoader를 생성한다.
  train_data_loader = DataLoader(
    dataset=train_dataset,
    batch_size=32,
    shuffle=True,
    drop_last=True
  )

  # DataLoader를 통해 배치 단위로 데이터를 순회한다.
  for idx, batch in enumerate(train_data_loader):
    input, target = batch
    print(f"Batch {idx} - Input Shape: {input.shape}, Target Shape: {target.shape}")

Data Size: 20640, Input Shape: torch.Size([20640, 8])
Split sizes: 14448 4128 2064
Batch 0 - Input Shape: torch.Size([32, 8]), Target Shape: torch.Size([32, 1])
Batch 1 - Input Shape: torch.Size([32, 8]), Target Shape: torch.Size([32, 1])
Batch 2 - Input Shape: torch.Size([32, 8]), Target Shape: torch.Size([32, 1])
Batch 3 - Input Shape: torch.Size([32, 8]), Target Shape: torch.Size([32, 1])
Batch 4 - Input Shape: torch.Size([32, 8]), Target Shape: torch.Size([32, 1])
Batch 5 - Input Shape: torch.Size([32, 8]), Target Shape: torch.Size([32, 1])
Batch 6 - Input Shape: torch.Size([32, 8]), Target Shape: torch.Size([32, 1])
Batch 7 - Input Shape: torch.Size([32, 8]), Target Shape: torch.Size([32, 1])
Batch 8 - Input Shape: torch.Size([32, 8]), Target Shape: torch.Size([32, 1])
Batch 9 - Input Shape: torch.Size([32, 8]), Target Shape: torch.Size([32, 1])
Batch 10 - Input Shape: torch.Size([32, 8]), Target Shape: torch.Size([32, 1])
Batch 11 - Input Shape: torch.Size([32, 8]), Target Shape:

Dataset 클래스로 모든 전처리 과정을 캡슐화함으로써, 메인 코드에서는 CaliforniaHousingDataset() 한 줄로 모든 준비가 완료된 데이터셋을 생성할 수 있다.

이후 DataLoader와 random_split을 사용하는 방식은 다른 데이터셋의 경우와 동일하게 적용된다.

8. 시계열 데이터 로더 

In [51]:
import os
from pathlib import Path

import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader, random_split

# 주피터 환경에서 __file__ 변수가 없어 발생하는 오류를 해결하기 위한 코드
current_path = Path(os.getcwd()) 
BASE_PATH = str(current_path.parent) # 현재 경로의 상위 폴더를 기본 경로로 설정
import sys
sys.path.append(BASE_PATH)


class BikesDataset(Dataset):
  def __init__(self, train=True, test_days=1):
    self.train = train
    self.test_days = test_days

    # 1. 데이터 로딩 및 기본 전처리
    bikes_path = os.path.join(BASE_PATH, "_00_data", "e_time-series-bike-sharing-dataset", "hour-fixed.csv")

    bikes_numpy = np.loadtxt(
      fname=bikes_path, dtype=np.float32, delimiter=",", skiprows=1,
      converters={
        1: lambda x: float(x[8:10])  # 날짜 문자열에서 '일'만 추출
      }
    )
    bikes = torch.from_numpy(bikes_numpy)
    daily_bikes = bikes.view(-1, 24, bikes.shape[1])  # 시간별 -> 일별 데이터로
    self.daily_bikes_target = daily_bikes[:, :, -1].unsqueeze(dim=-1) # 타겟 분리
    self.daily_bikes_data = daily_bikes[:, :, :-1] # 데이터 분리

    # 3. 범주형 데이터 원-핫 인코딩
    eye_matrix = torch.eye(4)
    day_data_torch_list = []
    for daily_idx in range(self.daily_bikes_data.shape[0]):
      day = self.daily_bikes_data[daily_idx]
      weather_onehot = eye_matrix[day[:, 9].to(torch.int64) - 1] # 날씨 정보는 1부터 시작하므로 인덱싱을 위해 1을 빼줌
      day_data_torch = torch.cat(tensors=(day, weather_onehot), dim=1)
      day_data_torch_list.append(day_data_torch)
    self.daily_bikes_data = torch.stack(day_data_torch_list, dim=0)

    # 4. 불필요한 특성 제거
    self.daily_bikes_data = torch.cat(
      [self.daily_bikes_data[:, :, :9], self.daily_bikes_data[:, :, 10:]], dim=2
    )
    # 5. 시계열 데이터 분할 (Train/Test)
    total_length = len(self.daily_bikes_data)
    self.train_bikes_data = self.daily_bikes_data[:total_length - test_days]
    self.train_bikes_targets = self.daily_bikes_target[:total_length - test_days]
    self.test_bikes_data = self.daily_bikes_data[-test_days:]
    self.test_bikes_targets = self.daily_bikes_target[-test_days:]

    # 6. 정규화 (학습 데이터 기준으로)
    train_temperatures = self.train_bikes_data[:, :, 9]
    train_temperatures_mean = torch.mean(train_temperatures)
    train_temperatures_std = torch.std(train_temperatures)
    # 학습 데이터의 평균/표준편차를 사용하여 학습 데이터와 테스트 데이터를 모두 정규화
    self.train_bikes_data[:, :, 9] = (self.train_bikes_data[:, :, 9] - train_temperatures_mean) / train_temperatures_std
    self.test_bikes_data[:, :, 9] = (self.test_bikes_data[:, :, 9] - train_temperatures_mean) / train_temperatures_std

    assert len(self.train_bikes_data) == len(self.train_bikes_targets)
    assert len(self.test_bikes_data) == len(self.test_bikes_targets)

  def __len__(self):
    return len(self.train_bikes_data) if self.train is True else len(self.test_bikes_data)

  def __getitem__(self, idx):
    if self.train is True:
      return self.train_bikes_data[idx], self.train_bikes_targets[idx]
    else:
      return self.test_bikes_data[idx], self.test_bikes_targets[idx]
        
  def __str__(self):
    if self.train is True:
      str = "Data Size: {0}, Input Shape: {1}, Target Shape: {2}".format(
        len(self.train_bikes_data), self.train_bikes_data.shape, self.train_bikes_targets.shape
      )
    else:
      str = "Data Size: {0}, Input Shape: {1}, Target Shape: {2}".format(
        len(self.test_bikes_data), self.test_bikes_data.shape, self.test_bikes_targets.shape
      )
    return str



os.getcwd()와 pathlib.Path를 조합하여 Jupyter Notebook 환경에서도 프로젝트의 기본 경로(BASE_PATH)를 유연하게 설정했다.

np.loadtxt의 converters 인자는 특정 열에 사용자 정의 함수를 적용하여, 파일을 불러오는 동시에 간단한 전처리를 수행하게 한다.
view(-1, 24, ...)는 시간 단위 데이터를 24시간(하루) 기준으로 묶어주는 시계열 데이터 재구성의 핵심적인 기법이다.

torch.eye로 단위 행렬을 만든 뒤, 범주 값을 인덱스로 사용하여 원-핫 벡터를 효율적으로 생성하고 torch.cat으로 기존 데이터에 합쳤다.
시계열 데이터에서는 미래 데이터가 학습에 사용되는 것을 막기 위해 데이터를 시간 순으로 잘라 학습셋과 테스트셋을 구성한다.

테스트 데이터의 정보가 사전에 노출되는 데이터 누수(Data Leakage)를 방지하기 위해, 정규화에 필요한 평균, 표준편차 등의 통계치는 오직 학습 데이터에서만 계산하여 모두에게 적용한다.
__init__에서 미리 준비해 둔 self.train_bikes_data와 self.test_bikes_data를 train 플래그를 이용해 선택적으로 반환하여, 하나의 클래스로 두 가지 목적의 데이터셋을 제공한다.

In [52]:
if __name__ == "__main__":
  # 학습 및 검증 데이터셋 준비
  train_bikes_dataset = BikesDataset(train=True, test_days=1)
  print(train_bikes_dataset)
  train_dataset, validation_dataset = random_split(train_bikes_dataset, [0.8, 0.2])
  train_data_loader = DataLoader(dataset=train_dataset, batch_size=32, shuffle=True, drop_last=True)
  validation_data_loader = DataLoader(dataset=validation_dataset, batch_size=32)

  # 테스트 데이터셋 준비
  test_dataset = BikesDataset(train=False, test_days=1)
  print(test_dataset)
  test_data_loader = DataLoader(dataset=test_dataset, batch_size=len(test_dataset))


Data Size: 729, Input Shape: torch.Size([729, 24, 19]), Target Shape: torch.Size([729, 24, 1])
Data Size: 1, Input Shape: torch.Size([1, 24, 19]), Target Shape: torch.Size([1, 24, 1])


BikesDataset(train=True)로 학습용 데이터 풀을 먼저 생성하고, random_split을 이용해 이 풀 안에서 최종 학습셋과 검증셋으로 나눈다.

BikesDataset(train=False)로 테스트셋을 별도로 생성하여, 학습 및 검증 과정과 완전히 분리시킨다.

실제 이미지 데이터를 텐서로 다루는 방법을 배우며, 딥러닝에서 데이터 전처리가 얼마나 중요한지 알 수 있었다. 특히 라이브러리마다 이미지 데이터의 차원 순서(HWC, CHW)가 다르다는 점과, 이를 permute를 통해 맞춰주는 과정에서 shape을 다루는 능력이 핵심이라는 것을 다시 한번 느끼게 되는 숙제였던 것 같다.
파일 경로를 맞추는 데서 시간을 조금 썼습니다. 스크립트 실행 위치에 따라 상대 경로가 달라져서 FileNotFoundError를 마주했는데, 이런 환경 설정 문제가 종종 발목을 잡는 것 같습니다.
1번 숙제에서 permute나 view 같은 함수들을 이론적으로 배웠을 때는 추상적으로 느껴졌는데, 2번 숙제에서 바로 실제 이미지에 적용해보니 개념이 확실히 연결되었다. 
이론과 실습을 겸하는 수업 방식 덕분에 이해가 잘 되는 것 같아 좋았다. 
좋은 수업 항상 감사합니다!